In [1]:
!pip install spacy --quiet

In [2]:
import spacy
from spacy.lang.en.examples import sentences
import pandas as pd

In [3]:
!python -m spacy download en_core_web_sm --quiet

[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [4]:
nlp = spacy.load('en_core_web_sm')
doc = nlp(sentences[0])
doc.text

'Apple is looking at buying U.K. startup for $1 billion'

In [5]:
from pprint import pprint as pp

def split_tokens(doc):
    cols = ['text', 'lemma_', 'pos_', 'tag_', 'dep_', 'is_alpha', 'is_stop']
    df = pd.DataFrame(columns=cols)

    for token in doc:
        # print(f'{token.text} {token.lemma_} {token.pos_} {token.tag_} {token.dep_} {token.is_alpha} {token.is_stop}')
        df = df.append(pd.Series(
            [token.text,
            token.lemma_,
            token.pos_, 
            token.tag_, 
            token.dep_, 
            token.is_alpha, 
            token.is_stop
            ], index=cols),ignore_index=True)
    return df

split_tokens(doc)

,text,lemma_,pos_,tag_,dep_,is_alpha,is_stop
0,Apple,Apple,PROPN,NNP,nsubj,True,False
1,is,be,AUX,VBZ,aux,True,True
2,looking,look,VERB,VBG,ROOT,True,False
3,at,at,ADP,IN,prep,True,True
4,buying,buy,VERB,VBG,pcomp,True,False
5,U.K.,U.K.,PROPN,NNP,dobj,False,False
6,startup,startup,VERB,VBD,dep,True,False
7,for,for,ADP,IN,prep,True,True
8,$,$,SYM,$,quantmod,False,False
9,1,1,NUM,CD,compound,False,False


In [6]:
comment="Mustafa, The datasets are interesting, but I am not sure how you will connect it to the road conditions. You also shared a very specific case which again I am not sure how can be obtained from the data. What you can do instead is to look how traffic accidents and violations relates. Do violations require a police presence? Then you can estimate when police was present and it was not. and that can be a reasoning to connect to accidents. Let me know what you think. Dmitry"

content = nlp(comment)
split_tokens(content)

,text,lemma_,pos_,tag_,dep_,is_alpha,is_stop
0,Mustafa,Mustafa,PROPN,NNP,nsubj,True,False
1,",",",",PUNCT,",",punct,False,False
2,The,the,DET,DT,det,True,True
3,datasets,dataset,NOUN,NNS,nsubj,True,False
4,are,be,AUX,VBP,ROOT,True,True
...,...,...,...,...,...,...,...
92,what,what,PRON,WP,dobj,True,True
93,you,you,PRON,PRP,nsubj,True,True
94,think,think,VERB,VBP,ccomp,True,False
95,.,.,PUNCT,.,punct,False,False
